In [4]:
!pip --version

pip 20.0.2 from /usr/lib/python3/dist-packages/pip (python 3.8)


In [5]:
!pip install trivial-torch-tools

In [6]:
import torch.nn.functional as F
import torch, torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [7]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [8]:
from trivial_torch_tools import Sequential
from collections import OrderedDict
globals()["default_device"] = torch.device("cpu")
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        layers = Sequential(input_shape = (1,28,28)) # given the input shape size
        layers.add_module('conv1', nn.Conv2d(1, 10, kernel_size=5))
        layers.add_module('conv1_pool', nn.MaxPool2d(2))
        layers.add_module('relu', nn.ReLU())
        layers.add_module('conv2', nn.Conv2d(10, 10, kernel_size=5))
        layers.add_module('conv2_pool', nn.MaxPool2d(2))
        layers.add_module('relu2', nn.ReLU())
        layers.add_module("conv2_drop", nn.Dropout2d())
        layers.add_module('flatten', nn.Flatten(1)) 
        layers.add_module('fc1', nn.Linear(layers.output_size, 50))
        layers.add_module('relu3', nn.ReLU())
        layers.add_module('fc2', nn.Linear(layers.output_size, 10)) 
        layers.add_module('softmax', nn.LogSoftmax())
        self.layers = layers

    def forward(self, x):
        return self.layers(x)

In [9]:

network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [10]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [11]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './files/model.pth')
      torch.save(optimizer.state_dict(), './files/optimizer.pth')

In [12]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [13]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/home/dhe/.local/lib/python3.8/site-packages/trivial_torch_tools/model.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return functools.reduce(
/home/dhe/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3096, Accuracy: 1043/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297891
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.314451
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.282737
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.248799
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.197456
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.234931
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.200129
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.067367
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.040800
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.975181
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.843496
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.707868
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.632557
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.521806
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.329429
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.275666
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.172144
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.322632
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.075748
Train Epoch: 1 [12160